In [ ]:
# Python Imaging Library (PIL)의 Image 모듈을 불러옴. 이미지 파일을 열고 조작하는 데 사용함.
from PIL import Image
# Matplotlib의 pyplot 모듈을 plt 별칭으로 불러옴. 시각화에 사용함.
import matplotlib.pyplot as plt
# PyTorch 핵심 라이브러리를 불러옴.
import torch
# 신경망 레이어(nn) 모듈을 불러옴.
import torch.nn as nn
# 옵티마이저(optim) 모듈을 불러옴.
import torch.optim as optim
# TorchVision 라이브러리를 불러옴. (이미지 변환 및 데이터셋 등에 사용)
import torchvision
import matplotlib.pyplot as plt

# 학습에 사용할 장치(Device)를 설정함. CUDA(GPU)가 사용 가능하면 'cuda'를, 아니면 'cpu'를 선택함.
dvc = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dvc

In [ ]:
# pil -> tensor
def image_to_tensor(image_filepath, image_dimension=128):
    img = Image.open(image_filepath).convert('RGB') # BGR -> RGB
    # 디버깅 : pil -> plt 출력
    plt.figure()
    plt.imshow(img)
    plt.show()
    # 이미지 해상도별 조정
    if max(img.size) <= image_dimension:
        img_size = max(img.size)
    else:
        img_size = image_dimension
    # tensor pipeline
    torch_transform = torchvision.transforms.Compose([
        torchvision.transforms.Resize(img_size),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    # 변환 적용
    img = torch_transform(img).unsqueeze(0)

    return img.to(dvc, torch.float)

style_image = image_to_tensor('vangogh_starry_night.jpg')
content_image = image_to_tensor('Tuebingen_Neckarfront.jpg')


In [ ]:
def gram_matrix(ip):
    n_batch, n_channel, h, w = ip.size()
    feats = ip.view(n_batch, n_channel, h * w)
    gram_mat = torch.bmm(feats, feats.transpose(1,2))
    return gram_mat.div(n_batch * n_channel * h * w) 
    

In [ ]:
# feature 부분 확인 및 출력
vgg19_model = torchvision.models.vgg19(pretrained=True).to(dvc)
vgg19_model = vgg19_model.features
print(vgg19_model)

# freeze
for p in vgg19_model.parameters():
  p.requires_grad = False

In [ ]:
conv_indices = []

for i in range(len(vgg19_model)):
    # MaxPool2d를 AvgPool2d로 변경
    if vgg19_model[i]._get_name() == 'MaxPool2d':
        vgg19_model[i]  = nn.AvgPool2d(kernel_size=vgg19_model[i].kernel_size,
                                   stride = vgg19_model[i].stride,
                                   padding=vgg19_model[i].padding)
    if vgg19_model[i]._get_name() == 'Conv2d':
        conv_indices.append(i)

conv_indices = dict(enumerate(conv_indices, 1)) # {1:0, 2:2, 3:5, ...}


In [ ]:
# clip until the last relevant layer
layers = {1:'s', 2:'s', 3:'s', 4:'sc', 5:'s'}
# vgg feature extractor -> nn.ModuleList
# - list : 인덱싱, 슬라이싱 가능
# - <> nn.Sequential 불가
# Avgpool 설정 : 모듈 block화 -> 출력
vgg_layers = nn.ModuleList(vgg19_model)

# 디버깅: 가장 깊은 레이어 확인
last_layer_idx = conv_indices[max(conv_indices.keys())]

vgg_layers_trimmed = vgg_layers[:last_layer_idx+1] # vgg_layers 보존
neural_style_transfer_model = nn.Sequential(*vgg_layers_trimmed) # 언패킹

# * : 언패킹 연산자

In [ ]:
neural_style_transfer_model

In [ ]:
vgg_layers

In [ ]:
# 두 가지 테스트 방향
# 1. 최적화 대상 이미지를 콘텐츠 이미지와 동일하게 초기화
# ip_image = content_image.clone()

# 2 선택
# 2. 최적화 대상(ip_image)를 콘텐츠 이미지 크기와 동일한 랜덤 노이즈로 초기화
ip_image = torch.randn(content_image.size(), device=dvc)

# 초기화된 노이즈 시각화
plt.figure()
# 텐서에서 배치차원(0번)제거, 기울기 비활성화, 넘파이로 변환, 채널 순서
plt.imshow(ip_image.squeeze(0).cpu().detach().numpy().transpose(1,2,0).clip(0,1))
plt.show()

In [ ]:
n_epoch = 300
# 손실 가중치 설정 (스타일 손실을 콘텐츠 손실보다 훨씬 크게 설정)
wt_style = 1e6 # GramMatrix 기반 -> 값의 크기가 작음
wt_content = 1

style_losses, content_losses = [], []
opt = optim.Adam([ip_image.requires_grad_()], lr=0.1)

for curr_epoch in range(1, n_epoch+1):
    ip_image.data.clamp_(0, 1) # 생성된 이미지 자름
    opt.zero_grad()
    
    # 누적 손실변수 초기화
    epoch_style_loss, epoch_content_loss = 0, 0

    for k in layers.keys():
        if 'c' in layers[k]:
            # 콘텐츠 이미지의 '특징추출' 진행, 변화계산 제외
            target = neural_style_transfer_model[:conv_indices[k]+1](content_image).detach()
            # 현재 이미지 특징추출
            ip = neural_style_transfer_model[:conv_indices[k]+1](ip_image)
            # 콘텐츠 손실 계산, 누적
            epoch_content_loss += torch.nn.functional.mse_loss(ip, target)

        if 's' in layers[k]:
            # 스타일 이미지의 'gram matrix 계산' 진행, 변화계산 제외
            target = gram_matrix(neural_style_transfer_model[:conv_indices[k]+1](style_image)).detach()
            # 현재 이미지 특징추출
            ip = gram_matrix(neural_style_transfer_model[:conv_indices[k]+1](ip_image))
            # 스타일 손실 계산, 누적
            epoch_style_loss += torch.nn.functional.mse_loss(ip, target)

    # 누적 스타일 손실에 가중치 반영
    epoch_style_loss *= wt_style
    # 누적 컨텐츠 손실에 가중치 반영
    epoch_content_loss *= wt_content
    # 최종 손실 계산
    total_loss = epoch_style_loss + epoch_content_loss
    # 최종 손실 역전파
    total_loss.backward()

    # 50 에폭마다 현재 상태를 출력하고 시각화함.
    if curr_epoch % 50 == 0:
        print(f"epoch number {curr_epoch}")
        print(f"style loss = {epoch_style_loss}, content loss = {epoch_content_loss}")
        plt.figure()
        plt.title(f"epoch number {curr_epoch}")
        # 생성된 이미지를 역변환(clamp, squeeze, numpy, transpose)하여 시각화함.
        plt.imshow(ip_image.data.clamp_(0, 1).squeeze(0).cpu().detach().numpy().transpose(1,2,0))
        plt.show()
        # 손실 값을 리스트에 저장함.
        style_losses.append(epoch_style_loss.item())
        content_losses.append(epoch_content_loss.item())

    # 옵티마이저를 사용하여 변화도 방향으로 이미지 픽셀 값을 업데이트함.
    opt.step()


In [ ]:
plt.plot(range(50, 300+1, 50), style_losses, label='style_loss')
plt.plot(range(50, 300+1, 50), content_losses, label='content_loss')
plt.legend()
plt.show()